# utility functions

In [1]:
def make_undirected(G):
    """Makes directed graph undirected by adding reverse edges."""
    V,E = G
    reverse_edges = []
    for (u,v) in E:
        if (v,u) not in E and (v,u) not in reverse_edges:
            reverse_edges.append((v,u))
    return (V, E+reverse_edges)

# path

In [3]:
import collections
def path(G,s,t):
    """Check if there is a path from node s to node t in graph G."""
    V,E = G
    visited = []
    queue = collections.deque([s])
    while queue: # |\label{bfs:line-outer-loop}|
        node = queue.popleft()
        if node == t:
            return True
        if node not in visited:
            visited.append(node)
            node_neighbors = [ v for (u,v) in E if u==node ] # |\label{bfs:line-find-neighbors}|
            for neighbor in node_neighbors: # |\label{bfs:line-inner-loop}|
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

In [5]:
V = [1,2,3,4,5,6,7,8]
E = [ (1,2), (2,3), (3,4), (4,5), (6,7), (7,8), (8,6) ]
G = (V,E)
print "path from {} to {}? {}".format(4, 1, path(G, 4, 1))
print "path from {} to {}? {}".format(1, 4, path(G, 1, 4))

G = make_undirected(G)
print "path from {} to {}? {}".format(4, 1, path(G, 4, 1))
print "path from {} to {}? {}".format(1, 4, path(G, 1, 4))
print "path from {} to {}? {}".format(4, 7, path(G, 4, 7))

path from 4 to 1? False
path from 1 to 4? True
path from 4 to 1? True
path from 1 to 4? True
path from 4 to 7? False


# relatively prime

In [6]:
def gcd(x,y):
    """Euclid's algorithm for greatest common divisor."""
    while y>0:
        x = x % y
        x,y = y,x
    return x

def rel_prime(x,y):
    return gcd(x,y) == 1

In [7]:
print gcd(24,60), rel_prime(24,60)
print gcd(25,63), rel_prime(25,63)

12 False
1 True


# connected

In [8]:
import itertools
def connected(G):
    """Check if G is connected."""
    V,E = G
    for (s,t) in itertools.combinations(V,2):
        if not path(G,s,t):
            return False
    return True

In [9]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = make_undirected(G)
print connected(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6), (2,5) ]
G = (V,E)
G = make_undirected(G)
print connected(G)

False
True


# Eulerian path

In [11]:
def degree(node, E):
    """Return degree of node."""
    return sum(1 for (u,v) in E if u==node)

def eulerian_path(G):
    """Check if G has an Eulerian path."""
    V,E = G
    num_deg_odd = 0
    V_pos = [] # nodes with positive degree
    for u in V:
        deg = degree(u, E)
        if deg % 2 == 1:
            num_deg_odd += 1
        if deg > 0:
            V_pos.append(u)
    if num_deg_odd not in [0,2]:
        return False
    G_pos = (V_pos,E)
    return connected(G_pos)

In [12]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = make_undirected(G)
print eulerian_path(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,1) ]
G = (V,E)
G = make_undirected(G)
print eulerian_path(G)

False
True


# Hamiltonian path verifier

In [13]:
def ham_path_verify(G, p):
    """Verify that p is a Hamiltonian path in G."""
    V,E = G
    # verify each pair of adjacent nodes in p shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    # verify p has |V| nodes
    if len(p) != len(V):
        return False
    # verify p has no duplicates
    if len(set(p)) != len(p):
        return False
    return True

In [14]:
V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)

p_bad = [1,2,3,4,5]
print ham_path_verify(G, p_bad)

p_bad2 = [1,2,4,3,1,2,4,3,5]
print ham_path_verify(G, p_bad2)

p_good = [1,2,4,3,5]
print ham_path_verify(G, p_good)

False
False
True


# composite verifier

In [2]:
def composite_verify(n,p):
    """Verify that p is a nontrivial divisor of n."""
    if not 1 < p < n:
        return False
    return n % p == 0 

In [6]:
print composite_verify(15, 3)
print composite_verify(15, 4)
                       
for p in range(1,20):
    print composite_verify(17, p), 
print

for p in range(1,20):
    print composite_verify(18, p), 

True
False
False False False False False False False False False False False False False False False False False False False
False True True False False True False False True False False False False False False False False False False


# clique verifier

In [13]:
import itertools
def clique_v(G, k, C):
    """Verify C is a k-clique in G."""
    V,E = G
    # verify C is the correct size
    if len(C) != k:
        return False
    # verify each pair of nodes in C shares an edge
    for (u,v) in itertools.combinations(C, 2):
        if (u,v) not in E:
            return False
    return True

In [15]:
V = [1,2,3,4,5,6]
E = [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,5), (5,6), (4,6)]
G = (V,E)
G = make_undirected(G)

C = [1,2,3,4]
k = len(C)
print '{} is a {}-clique in G? {}'.format(C, k, clique_v(G, k, C))

C = [3,4,5]
print '{} is a {}-clique in G? {}'.format(C, k, clique_v(G, k, C))

C = [1,3,4,5]
print '{} is a {}-clique in G? {}'.format(C, k, clique_v(G, k, C))

[1, 2, 3, 4] is a 4-clique in G? True
[3, 4, 5] is a 4-clique in G? False
[1, 3, 4, 5] is a 4-clique in G? False


# subset sum verifier

In [17]:
import itertools
def subset_sum_v(C, t, S):
    """Verifies that S is a subcollection of C summing to t."""
    if sum(S) != t: # check sum
        return False
    C = list(C) # make copy that we can change
    for n in S: # ensure S is a subcollection of C
        if n not in C:
            return False
        C.remove(n)
    return True

In [18]:
C = [1,2,3,4,5,6,7,8]
t = 20
S = [1,2,3,4,5,6]
print subset_sum_v(C, t, S)

C = [1,2,3,4,5,6,7,8]
t = 20
S = [8, 7, 5]
print subset_sum_v(C, t, S)

False
True


# NTM

In [19]:
import collections

def next_configs(delta, config):
    """Apply delta to configuration (state, pos, content)
    to get list of possible next configurations."""
    state, pos, content = config # give names to each part of triple
    configs = []
    symbol = content[pos]
    for (new_state, new_symbol, move) in delta(state, symbol):
        new_content = content[:pos] + new_symbol + content[pos+1:]
        new_pos = max(0, pos + move)
        if new_pos >= len(new_content):
            new_content += "-"
        config = (new_state, new_pos, new_content)
        configs.append(config)
    return configs

def M(x):
    """An NTM that replaces its input with a
    nondeterministically chosen string of the same length
    accepts if it ever guessed a 1, otherwise rejects"""
    init_config = ("q0", 0, x+"_")
    def delta(state, symbol):
        # if in halting state, no next configuration
        if state in ["qA", "qR"]:
            return []
        if symbol == "_":  # if blank, halt
            if state == "q1":
                return [("qA", "_", -1)]
            elif state == "q0":
                return [("qR", "_", -1)]
        else: 
            # write a 0 or 1 over the current symbol
            # remember if a 1 is written
            if state == "q0":
                return [ ("q0", "0", 1), ("q1", "1", 1) ]
            elif state == "q1":
                return [ ("q1", "0", 1), ("q1", "1", 1) ]
        assert False

    # do breadth-first search of configuration graph
    visited = []
    queue = collections.deque([init_config])
    while queue:
        config = queue.popleft()
        if config[0] == "qA":
            return True
        if config not in visited:
            visited.append(config)
            # get all possible next configurations; see below
            neighbors = next_configs(delta, config)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

In [20]:
M('0101')

True

# Hamiltonian path NTM

In [22]:
import random

def ham_path_ntm(G):
    """ 'NTM' (implemented via random) for deciding HamPath. """
    V,E = G
    # guess a path p that visits each node exactly once
    p = list(V)
    random.shuffle(p)
    # verify each pair of adjacent nodes in pi shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    print p, 
    return True

In [23]:
V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)
for _ in range(500):
    print ham_path_ntm(G), 

False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False [1, 2, 4, 3, 5] True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False [1, 2, 4, 3, 5] True False False False False False False False False False False False False [1, 2, 4, 3, 5] True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False F

# general exponential-time algorithm for finding NP witnesses

In [ ]:
import itertools 

def V_A(x,w):
    """Verifier for decision problem A."""
    raise NotImplementedError()

def binary_strings_of_length(length):
    """A nice Python way to generate all strings of a given length."""
    return map(lambda lst: ''.join(lst), itertools.product(['0', '1'], repeat=length))

def A_decider(x):
    """Exponential-time algorithm for finding witnesses."""
    n = len(x)
    for m in range(n**c + 1): # check lengths m in [0,1,...,n^c]
        for w in binary_strings_of_length(m):
            if V_A(x,w): # assumes V_A is implemented
                return True
    return False

# Hamiltonian path exponential-time deterministic algorithm

In [16]:
import itertools

def ham_path_brute_force(G):
    """Exponential-time algorithm for finding Hamiltonian paths,
    which calls the verifier on all potential witnesses."""
    V,E = G
    for p in itertools.permutations(V):
        if ham_path_verify(G, p):
            return True
    return False

In [17]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
print ham_path_brute_force(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,5), (5,6) ]
G = (V,E)
print ham_path_brute_force(G)

False
True


# Boolean formulas

In [118]:
class Boolean_formula(object):
    """Represents a Boolean formula with AND, OR, and NOT operations."""
    def __init__(self, variable=None, op=None, phi=None, psi=None):
        if not ( (variable == None and op == "not" and phi != None and psi == None) or 
                 (variable == None and op == "and" and phi != None and psi != None) or 
                 (variable == None and op == "or"  and phi != None and psi != None) or 
                 (variable != None and op == phi == psi == None) ):
            raise ValueError("Must either set variable for base case" +\
                             "or must set op to 'not', 'and', or 'or'" +\
                             "and recursive formulas phi and psi")
        self.variable = variable
        self.op = op
        self.phi = phi
        self.psi = psi
        if self.variable:
            self.variables = [variable]
        elif op == 'not':
            self.variables = phi.variables
        elif op in ['and', 'or']:
            self.variables = list(phi.variables)
            self.variables.extend(x for x in psi.variables if x not in phi.variables)
            self.variables.sort()
        
    def evaluate(self, assignment):
        """Value of this formula with given assignment,  a dict mapping variable names to Python booleans.
        
        Assignment can also be a string of bits, which will be mapped to variables in alphabetical order.
        Boolean values are interconverted with integers to make for nicer printing 
        (0 and 1 versus True and False)"""
        if type(assignment) is str:
            assignment = dict(zip(self.variables, map(int, assignment)))
        if self.op == None:
            return assignment[self.variable]
        elif self.op == 'not':
            return int(not self.phi.evaluate(assignment))
        elif self.op == 'and':
            return int(self.phi.evaluate(assignment) and self.psi.evaluate(assignment))
        elif self.op == 'or':
            return int(self.phi.evaluate(assignment) or self.psi.evaluate(assignment))
        else:
            raise ValueError("This shouldn't be reachable")

In [121]:
# verbose way to build formula (x and y) or (z and not y)
# since I don't feel like writing a parser for Boolean formulas
# if I want to do that in the future, go here:
#   http://pyparsing.wikispaces.com/file/view/simpleBool.py
x = Boolean_formula(variable="x")
y = Boolean_formula(variable="y")
z = Boolean_formula(variable="z")
x_and_y = Boolean_formula(op="and", phi=x, psi=y)
not_y = Boolean_formula(op="not", phi=y)
z_and_not_y = Boolean_formula(op="and", phi=z, psi=not_y)
formula = Boolean_formula(op="or", phi=x_and_y, psi=z_and_not_y)

import itertools
num_variables = len(formula.variables)
for assignment in itertools.product(["0","1"], repeat=num_variables):
    assignment = "".join(assignment)
    value = formula.evaluate(assignment)
    print "formula value = {} on assignment {}".format(value, assignment)

formula value = 0 on assignment 000
formula value = 1 on assignment 001
formula value = 0 on assignment 010
formula value = 0 on assignment 011
formula value = 0 on assignment 100
formula value = 1 on assignment 101
formula value = 1 on assignment 110
formula value = 1 on assignment 111


# polynomial-time mapping reduction of Clique to Independent-Set

In [18]:
import itertools
def reduction_from_clique_to_independent_set(G):
    V,E = G
    Ep = [ {u,v} for (u,v) in itertools.combinations(V,2) if {u,v} not in E and u!=v ]
    Gp = (V,Ep)
    return Gp

In [19]:
from pprint import pprint
V = [1,2,3,4,5,6,7]
E = [ {1,2}, {2,3}, {3,4}, {1,5}, {5,6}, {6,7}, {2,5}, {2,6}, {3,5}, {3,6}, {3,7} ]
G = (V,E)
pprint(G)
pprint(reduction_from_clique_to_independent_set(G))

([1, 2, 3, 4, 5, 6, 7],
 [set([1, 2]),
  set([2, 3]),
  set([3, 4]),
  set([1, 5]),
  set([5, 6]),
  set([6, 7]),
  set([2, 5]),
  set([2, 6]),
  set([3, 5]),
  set([3, 6]),
  set([3, 7])])
([1, 2, 3, 4, 5, 6, 7],
 [set([1, 3]),
  set([1, 4]),
  set([1, 6]),
  set([1, 7]),
  set([2, 4]),
  set([2, 7]),
  set([4, 5]),
  set([4, 6]),
  set([4, 7]),
  set([5, 7])])


# using mapping reduction to show if B is in P, then A is in P

In [ ]:
def f(x):
    raise NotImplementedError()
    # TODO: code for f, reducing A to B, goes here

def M(y):
    raise NotImplementedError()
    # TODO: code for M, deciding B, goes here

def N(x):
    """Compose reduction f from A to B, with algorithm M for B,
    to get algorithm N for A."""
    y = f(x)
    output = M(y)
    return output

# enumerators

In [1]:
def is_prime(n):
    """Check if n is prime."""
    if n < 2: # 0 and 1 are not primes
        return False
    for x in xrange(2, int(n**0.5)+1):
        if n % x == 0:
            return False
    return True

import itertools
def primes_enumerator():
    """Iterate over all prime numbers."""
    for n in itertools.count(): #iterates over all natural numbers
        if is_prime(n):
            yield n

In [2]:
# print first 100 numbers returned from primes_enumerator()
for (p, count) in zip(primes_enumerator(), range(100)):
    print p,

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541
